# 🐷💫 Susie Package Astroplan Schedule Generation Example

## 🔵 Import the necessary python libraries and Susie objects.

Importing any packages and libraries needed, (Susie package coming directly from filesystem)

In [1]:
import numpy as np
import pandas as pd
from susie.timing_data import TimingData
from susie.ephemeris import Ephemeris

## 🔵 Import Data

Download the timing data (that includes occultations) from the GitHub repository

In [2]:
url = 'https://raw.githubusercontent.com/BoiseStatePlanetary/susie/main/example_data/wasp12b_tra_occ.csv'

# Read the CSV file directly from the URL
data = pd.read_csv(url)
tra_or_occs = np.array(data["tra_or_occ"])
epochs = np.array(data["epoch"].astype('int'))
mid_times = np.array(data["mid_time"])
mid_time_errs = np.array(data["mid_time_err"])

## 🔵 The Package

### 🔷 **STEP 1:** Add your transit and occultation data to the TimingData object.

In [3]:
# Create new transit times object with above data
timing_obj = TimingData('jd', epochs, mid_times, mid_time_uncertainties=mid_time_errs, tra_or_occ=tra_or_occs, time_scale='tdb')

### 🔷 **STEP 2:** Create the Ephemeris object and add your TimingData object.

In [4]:
ephemeris_obj = Ephemeris(timing_obj)

We'll use the quadratic model for this example. Get the model data dictionary from the ephemeris object.

In [5]:
quad_model_data = ephemeris_obj.get_model_ephemeris("quadratic")

Now we need two main objects to get the observing schedule
 1. The Observer object, which is created using the observer's Earth location (with longitude, latitude, and elevation if known)
 2. The Target object, which is created using the star's sky coordinates (with either the star's name or the right ascension and declination)

In [6]:
# Create observer object (at Boise State)
observer_obj = ephemeris_obj.create_observer_obj(timezone="US/Mountain", longitude=-116.208710, latitude=43.602,
                                                    elevation=821, name="BoiseState")

In [7]:
# Create target object (WASP-12)
target_obj = ephemeris_obj.create_target_obj("WASP-12")

Now we can call get observing schedule. For this we will need
   1. Model data dictionary from ouru desired ephemeris (we will use the quadratic ephemeris that we created above)
   2. A timezone (we will use US/Mountain for Boise State time)
   3. The observer object (created above)
   4. The target object (created above)
   5. The number of transits we want in our schedule
   6. The number of occultations we want in our schedule
   7. The observation start time, which will be the moment we start looking for observable transits/occultations
   8. And one of the two:
      - The name of the exoplanet, which will be used to get an eclipse duration OR
      - The eclipse duration itself if the object is either not a known exoplanet or you want to customize this parameter

In [8]:
ephemeris_obj.get_observing_schedule(quad_model_data, "US/Mountain", observer_obj, target_obj, 
                                     25, 25, "2024-01-07", "WASP-12 b")

,ingress,egress,type
5,2024-01-09 20:13:13 -0700,2024-01-09 23:13:16 -0700,occultation
6,2024-01-10 22:24:51 -0700,2024-01-11 01:24:55 -0700,occultation
7,2024-01-12 00:36:30 -0700,2024-01-12 03:36:34 -0700,occultation
0,2024-01-15 20:17:16 -0700,2024-01-15 23:17:19 -0700,transit
1,2024-01-16 22:28:54 -0700,2024-01-17 01:28:58 -0700,transit
2,2024-01-18 00:40:33 -0700,2024-01-18 03:40:37 -0700,transit
8,2024-01-21 20:21:18 -0700,2024-01-21 23:21:22 -0700,occultation
9,2024-01-22 22:32:57 -0700,2024-01-23 01:33:01 -0700,occultation
10,2024-01-24 00:44:36 -0700,2024-01-24 03:44:39 -0700,occultation
3,2024-01-27 20:25:21 -0700,2024-01-27 23:25:25 -0700,transit
